# Exercise: Create An Object Detection Solution

In this notebook, you wrap up the **Create An Object Detection Solution** exercise by using the `CustomVisionPredictionClient` object found in the Azure Cognitive Services Python SDK.

> **Important**: Several cells in this notebook contain `TODO` statements, where you need to update variables or enter code to enable the cell to execute without errors.

## Setup the notebook

### Install the Azure Cognitive Service Custom Vision Library

To access the Custom Vision service from this Python notebook, you need to install the Azure Cognitive Services Custom Vision Library. This library is part of the [Azure SDK for Python](https://github.com/Azure/azure-sdk-for-python) GitHub project.

> To learn more, read the [Azure Cognitive Services modules for Python](https://docs.microsoft.com/python/api/overview/azure/cognitive-services?view=azure-python) article in Microsoft Docs.

Execute the cell below to install the library.

In [ ]:
pip install azure-cognitiveservices-vision-customvision

### Import libraries and services

In addition to the Custom Vision library, we will also make use of a few other Python libraries in this notebook. Run the following cell to import the libraries and reference the services required to execute the cells below.

In [ ]:
# Import the libraries need to call the classification service
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os

## Set variables

To access your Custom Vision service, its `Project Id`, authentication `key` and `endpoint` URL need to be supplied to client applications.

### Retrieve and set the Project Id

`TODO`: Use the `Project Id` value found on the **Project Settings** page of the `Object Detector` project in the Custom Vision portal to replace the tokenized value for the `projectId` variable.

- Retrieve the `Project Id` value  to replace the tokenized value for the `projectId` variable below.

### Retrieve and set the `key` and `endpoint` values from your Custom Vision prediction service.

`TODO`: Using the values from the `Keys and Endpoints` page for you Custom Vision prediction service, replace the tokenized values in the cell below, as follows:

- Retrieve the `Key 1` value for your prediction cognitive services resource in the Azure portal and update the `key` value below.
- Retrieve the `Endpoint` value for your prediction cognitive services resource in the Azure portal and update the `endpoint` value below.

> **Important**: You **must** replace the variable values below with the values from your Custom Vision prediction endpoint or the remaining cells in this notebook will fail.

In [ ]:
# Update the values assigned to the variables below.
projectId = 'not authorized to expose'
key = 'not authorized to expose'
endpoint = 'not authorized to expose'

# The modelName variable's value must match the model name you set when you published the model!
modelName = 'object-detection-exercise'
print('Ready to detect objects using the model named "{}" from Custom Vision Project Id {}.'.format(modelName, projectId))

## Create a Custom Vision Prediction client

After setting the `projectId`, `key` and `endpoint` needed to access your Custom Vision Prediction service, you can instantiate a client.

`TODO`: In the cell below, locate the `TODO` statement and complete the `client = ` line to create a new Custom Vision Prediction Client object pointing to your Custom Vision prediction resource.

Once you have updated the code, execute the cell below to create a `CustomVisionPredictionClient` object.

In [ ]:
# Instantiate a Custom Vision Prediction client
credentials = ApiKeyCredentials(in_headers={"Prediction-key": key})
client = CustomVisionPredictionClient(endpoint=endpoint, credentials=credentials)

## Call your deployed object detection model

The cell below uses a Custom Vision client to send images to your custom object detection model and returns the predictions made by the model.

### Call the detect image method

`TODO`: In the cell below, locate the `TODO` statement (after `results = client.`) and complete the line of code to call the `Detect Image` method of the `CustomVisionPredictionClient` object.

> We covered this in the Object Detection demo, but if you need help, you can learn more by reading the [detect_image documentation](https://docs.microsoft.com/python/api/azure-cognitiveservices-vision-customvision/azure.cognitiveservices.vision.customvision.prediction.operations.customvisionpredictionclientoperationsmixin?view=azure-python#detect-image-project-id--published-name--image-data--application-none--custom-headers-none--raw-false----operation-config-).

After completing the `TODO` line, execute the cell below.

In [ ]:
%matplotlib inline

# Retrieve a test image from the 'test-images' folder
testImagePath = os.path.join('test-images', 'silverware-test-02.jpg')
testImage = Image.open(testImagePath)
testImgHeight, testImgWidth, testImgCh = np.array(testImage).shape

print('Detecting objects in {} using model {} in project {}...'.format(testImagePath, modelName, projectId))

# Identify each object in the test image
with open(testImagePath, mode="rb") as imageData:
    results = client.results = client.detect_image(projectId, modelName, imageData)
    
# Create a matplotlib figure to display the classification results
fig = plt.figure(figsize=(8, 8))
plt.axis('off')

# Output the image with bounding boxes assigned to each detected entity
draw = ImageDraw.Draw(testImage)
lineWidth = int(np.array(testImage).shape[1]/100)

object_colors = {
    "Fork": "lightblue",
    "Knife": "yellow",
    "Spoon": "orange"
}

for prediction in results.predictions:
    color = 'white' # default for 'other' object tags
    if (prediction.probability*100) > 50:
        if prediction.tag_name in object_colors:
            color = object_colors[prediction.tag_name]
        left = prediction.bounding_box.left * testImgWidth 
        top = prediction.bounding_box.top * testImgHeight 
        height = prediction.bounding_box.height * testImgHeight
        width =  prediction.bounding_box.width * testImgWidth
        points = ((left,top), (left+width,top), (left+width,top+height), (left,top+height),(left,top))
        draw.line(points, fill=color, width=lineWidth)
        plt.annotate(prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100),(left,top), backgroundcolor=color)
plt.imshow(testImage)